In [1]:
import os
import sys
import argparse
import copy
import pickle

import cv2
import xml.etree.ElementTree as et

import numpy as np

from openvino.inference_engine import IECore

In [2]:
MODEL_PATH = 'model/'

# Default shape

In [3]:
image = cv2.imread('dog.jpg')
img_height, img_width, img_channels = image.shape
image = cv2.resize(image, (224, 224))
image = image.transpose((2, 0, 1))

In [6]:
ie = IECore()
net = ie.read_network(model=MODEL_PATH + 'vgg16.xml') # Object detection model
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n, c, h, w = net.input_info[input_blob].input_data.shape
net.reshape({input_blob: (n, c, 200, 512)})

/tmp/ipykernel_113051/1676232401.py:3: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  input_blob = next(iter(net.inputs))


RuntimeError: Check 'input_product.get_length() % output_product.get_length() == 0' failed at core/src/op/reshape.cpp:331:
While validating node 'v1::Reshape fc6/flatten_fc_input (pool5[0]:f32{1,512,7,16}, fc6/flatten_fc_input/value[0]:i64{2}) -> (f32{?,?})' with friendly_name 'fc6/flatten_fc_input':
Non-'-1' output dimensions do not evenly divide the input dimensions


In [113]:
exec_net = ie.load_network(network=net, device_name='CPU')
res = exec_net.infer(inputs={input_blob: image})

In [114]:
t = (res[out_blob][0])

In [116]:
t

array([[[ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        ...,
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ]],

       [[ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        ...,
        [ 0.       ,  3.8551624,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [ 0.

# Reshape *2

In [9]:
net_reshape = ie.read_network(model=MODEL_PATH + 'vgg16.xml')
net_reshape.reshape({input_blob: (n, c, h*2, w*2)})

In [11]:
input_blob = next(iter(net_reshape.inputs))
out_blob = next(iter(net_reshape.outputs))
n, c, h, w = net_reshape.input_info[input_blob].input_data.shape
print(n, c, h, w)

1 3 448 448


/tmp/ipykernel_1361362/4231422558.py:1: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  input_blob = next(iter(net_reshape.inputs))


In [15]:
image2 = cv2.imread('dog.jpg')
image2 = cv2.resize(image2, (448, 448))
image2 = image2.transpose((2, 0, 1))


exec_net = ie.load_network(network=net_reshape, device_name='CPU')
res = exec_net.infer(inputs={input_blob: image2})

# Reshape /2

In [68]:
h, w = 224, 224
scale = 3

print(h * scale, w * scale)

672 672


In [69]:
net_reshape = ie.read_network(model=MODEL_PATH + 'vgg16.xml')
net_reshape.reshape({input_blob: (n, c, int(h*scale), int(w*scale))})

In [70]:
input_blob = next(iter(net_reshape.inputs))
out_blob = next(iter(net_reshape.outputs))
n, c, h, w = net_reshape.input_info[input_blob].input_data.shape
print(n, c, h, w)

1 3 672 672


/tmp/ipykernel_1361362/4231422558.py:1: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  input_blob = next(iter(net_reshape.inputs))


In [71]:
net_reshape.reshape({input_blob: (n, c, int(h/3), int(w/3))})

In [72]:
input_blob = next(iter(net_reshape.inputs))
out_blob = next(iter(net_reshape.outputs))
n, c, h, w = net_reshape.input_info[input_blob].input_data.shape
print(n, c, h, w)

1 3 224 224


/tmp/ipykernel_1361362/4231422558.py:1: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  input_blob = next(iter(net_reshape.inputs))


In [76]:
image3 = cv2.imread('dog.jpg')
w_, h_, c_ = image3.shape

In [77]:
w_, h_

(512, 512)

In [78]:
net_reshape = ie.read_network(model=MODEL_PATH + 'vgg16.xml')
net_reshape.reshape({input_blob: (n, c, w_, h_)})

RuntimeError: Check 'input_product.get_length() % output_product.get_length() == 0' failed at core/src/op/reshape.cpp:331:
While validating node 'v1::Reshape fc6/flatten_fc_input (pool5[0]:f32{1,512,16,16}, fc6/flatten_fc_input/value[0]:i64{2}) -> (f32{?,?})' with friendly_name 'fc6/flatten_fc_input':
Non-'-1' output dimensions do not evenly divide the input dimensions
